**well it is failed experiment for the time bieng**

In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Attention, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pickle
import os

# -----------------------------
# 1) Load dataset (JSONL)
# -----------------------------
data_path = "/content/dataset_t5.jsonl"  # <- your preprocessed file
input_texts, target_texts = [], []
with open(data_path, "r", encoding="utf-8") as f:
    for line in f:
        ex = json.loads(line)
        # Input: "fsm description: ..."
        input_texts.append(ex["input_text"])
        # Target: add start/end markers for decoder supervision
        target_texts.append("<start> " + ex["target_text"] + " <end>")

print(f"Samples: {len(input_texts)}")

# -----------------------------
# 2) Tokenization
#    (shared tokenizer for simplicity)
# -----------------------------
tokenizer = Tokenizer(filters="", lower=True, oov_token="<unk>")
tokenizer.fit_on_texts(input_texts + target_texts)
vocab_size = len(tokenizer.word_index) + 1
print("Vocab size:", vocab_size)

# Convert to sequences
enc_seqs = tokenizer.texts_to_sequences(input_texts)
dec_seqs = tokenizer.texts_to_sequences(target_texts)

encoder_input_data = pad_sequences(enc_seqs, padding="post")
decoder_input_data = pad_sequences(dec_seqs, padding="post")

# Create decoder targets by shifting left by 1
decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]
decoder_target_data[:, -1] = 0

max_encoder_len = encoder_input_data.shape[1]
max_decoder_len = decoder_input_data.shape[1]
print("Max input len:", max_encoder_len, " Max output len:", max_decoder_len)

# -----------------------------
# 3) Build ATTENTION Seq2Seq
# -----------------------------
latent_dim = 256
emb_dim = 128

# Encoder
encoder_inputs = Input(shape=(None,), name="encoder_inputs")
enc_emb = Embedding(vocab_size, emb_dim, name="enc_emb")(encoder_inputs)
# return sequences + states
encoder_lstm = LSTM(
    latent_dim, return_sequences=True, return_state=True,
    dropout=0.2, recurrent_dropout=0.0, name="encoder_lstm"
)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,), name="decoder_inputs")
dec_emb = Embedding(vocab_size, emb_dim, name="dec_emb")(decoder_inputs)
decoder_lstm = LSTM(
    latent_dim, return_sequences=True, return_state=True,
    dropout=0.2, recurrent_dropout=0.0, name="decoder_lstm"
)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# Attention (Luong-style via Keras Attention)
attn_layer = Attention(name="attention")
context = attn_layer([decoder_outputs, encoder_outputs])  # (batch, dec_len, latent)
concat = Concatenate(axis=-1, name="concat_ctx")([decoder_outputs, context])
concat = Dropout(0.2)(concat)

hidden = Dense(256, activation="tanh")(concat)
hidden = Dense(128, activation="tanh")(hidden)
decoder_dense = Dense(vocab_size, activation="softmax", name="output_dense")
final_outputs = decoder_dense(hidden)

# Training model
model = Model([encoder_inputs, decoder_inputs], final_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
model.summary()

# -----------------------------
# 4) Train (teacher forcing)
# -----------------------------
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
]

history = model.fit(
    [encoder_input_data, decoder_input_data],
    np.expand_dims(decoder_target_data, -1),
    batch_size=16,
    epochs=40,           # bump a bit for stability
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1
)

# -----------------------------
# 5) Save training artifacts
# -----------------------------
save_dir = "/content/attn_lstm_fsm"
os.makedirs(save_dir, exist_ok=True)
model_path = os.path.join(save_dir, "attn_lstm_fsm_model.h5")
tok_path = os.path.join(save_dir, "lstm_tokenizer.pkl")
model.save(model_path)
with open(tok_path, "wb") as f:
    pickle.dump(tokenizer, f)
print("✅ Saved:", model_path, "and", tok_path)

In [ ]:
"""
ye predictions ka lia ha and abhi iss say sahi ni aa rahi


# Load model & tokenizer if separate
from tensorflow.keras.models import load_model
import pickle

model = load_model("/content/lstm_fsm_model.h5")
with open("/content/lstm_tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

reverse_word_index = {v: k for k, v in tokenizer.word_index.items()}

def decode_sequence(input_text, max_output_len=100):
    # Encode input
    seq = tokenizer.texts_to_sequences([input_text])
    seq = pad_sequences(seq, maxlen=model.input[0].shape[1], padding='post')
    # Initialize states as zeros (simple decoding)
    states = None
    target_seq = np.array([[tokenizer.word_index['<start>']]])
    output_sentence = []
    for _ in range(max_output_len):
        preds = model.predict([seq, target_seq], verbose=0)
        pred_id = np.argmax(preds[0, -1, :])
        word = reverse_word_index.get(pred_id, "")
        if word == "<end>" or word == "":
            break
        output_sentence.append(word)
        target_seq = np.array([[pred_id]])
    return " ".join(output_sentence)

# 🔍 Test
example_input = "fsm description: odd number of 1's"
print("Generated:", decode_sequence(example_input))
"""

In [ ]:
# Build encoder inference model (reuses training tensors)
encoder_model = Model(encoder_inputs, [encoder_outputs, state_h, state_c])

# ----- Robustly fetch trained layers we need -----
# Decoder LSTM: we already have the layer object as `decoder_lstm` from training.

# Decoder Embedding layer: find it from the model (2nd Embedding typically)
emb_layers = [l for l in model.layers if isinstance(l, tf.keras.layers.Embedding)]
if len(emb_layers) >= 2:
    dec_emb_layer = emb_layers[1]
else:
    dec_emb_layer = emb_layers[-1]  # fallback

# Attention layer: grab the single Attention layer used during training
attn_layers = [l for l in model.layers if isinstance(l, tf.keras.layers.Attention)]
if not attn_layers:
    raise RuntimeError("Could not find Attention layer in the trained model.")
attn_layer_trained = attn_layers[0]

# Dense stack at the end: last three Dense layers in training were:
#   dense_hidden1 -> dense_hidden2 -> decoder_dense(softmax)
dense_layers = [l for l in model.layers if isinstance(l, tf.keras.layers.Dense)]
if len(dense_layers) < 3:
    raise RuntimeError("Expected at least 3 Dense layers (2 hidden + output) in the trained model.")
dense1, dense2, dense_out = dense_layers[-3], dense_layers[-2], dense_layers[-1]

# ----- Decoder inference graph (one token at a time) -----
# Inputs
inf_decoder_input = Input(shape=(1,), name="inf_decoder_input")  # previous token id
inf_state_h = Input(shape=(latent_dim,), name="inf_state_h")
inf_state_c = Input(shape=(latent_dim,), name="inf_state_c")
inf_encoder_outputs = Input(shape=(max_encoder_len, latent_dim), name="inf_encoder_outputs")

# Embed current token with the *trained* decoder embedding
inf_dec_emb = dec_emb_layer(inf_decoder_input)

# Run trained decoder LSTM one step with incoming states
inf_dec_outputs, out_h, out_c = decoder_lstm(inf_dec_emb, initial_state=[inf_state_h, inf_state_c])

# Apply the *trained* attention over encoder outputs
inf_context = attn_layer_trained([inf_dec_outputs, inf_encoder_outputs])

# Concatenate decoder step with its context, then pass through trained dense stack
inf_concat = Concatenate(axis=-1, name="inf_concat")([inf_dec_outputs, inf_context])
x = dense1(inf_concat)
x = dense2(x)
inf_token_probs = dense_out(x)  # softmax over vocab

decoder_model = Model(
    [inf_decoder_input, inf_state_h, inf_state_c, inf_encoder_outputs],
    [inf_token_probs, out_h, out_c]
)

import re
import numpy as np

# -------- helpers --------
def _clean_join(tokens):
    text = " ".join([t for t in tokens if t])
    return text.replace(" ,", ",").replace(" ;", ";").strip()

def _dedup_transitions(text):
    """
    Remove duplicate transitions while preserving order.
    Works whether your text has 'transitions:' header or just rules separated by ';'.
    """
    if "transitions:" in text:
        head, tail = text.split("transitions:", 1)
        items = [s.strip() for s in tail.split(";") if s.strip()]
        seen, out = set(), []
        for it in items:
            key = re.sub(r"\s+", "", it.lower())
            if key not in seen:
                seen.add(key)
                out.append(it)
        return (head + "transitions: " + "; ".join(out)).strip()
    else:
        items = [s.strip() for s in text.split(";") if s.strip()]
        seen, out = set(), []
        for it in items:
            key = re.sub(r"\s+", "", it.lower())
            if key not in seen:
                seen.add(key)
                out.append(it)
        return "; ".join(out)

def _no_repeat_ngram_ok(prev_ids, cand_id, n=3):
    if len(prev_ids) < n - 1:
        return True
    ctx = prev_ids[-(n-1):] + [cand_id]
    for i in range(len(prev_ids) - (n-1)):
        if prev_ids[i:i+n-1] + [cand_id] == ctx:
            return False
    return True

def _pick_with_constraints(probs, prev_ids, recent_window=30, repetition_penalty=1.2,
                           no_repeat_ngram=3, ban_ids=None, top_k=50):
    p = probs.copy().astype(np.float64) + 1e-12
    # repetition penalty on recent tokens
    if repetition_penalty > 1.0 and len(prev_ids) > 0:
        recent = set(prev_ids[-recent_window:])
        for rid in recent:
            if 0 <= rid < p.shape[-1]:
                p[rid] = p[rid] / repetition_penalty
    # ban ids (pad/start/unk)
    if ban_ids:
        for bid in ban_ids:
            if 0 <= bid < p.shape[-1]:
                p[bid] = 0.0
    # shortlist top-k
    top_k = min(top_k, p.shape[-1])
    top_ids = np.argpartition(-p, top_k-1)[:top_k]
    top_ids = top_ids[np.argsort(-p[top_ids])]
    # pick first that satisfies no-repeat-ngram
    for tid in top_ids:
        if no_repeat_ngram and not _no_repeat_ngram_ok(prev_ids, int(tid), n=no_repeat_ngram):
            continue
        return int(tid)
    return int(top_ids[0]) if len(top_ids) else int(np.argmax(p))

# -------- Constrained decoding + post-dedup --------
def generate_fsm(prompt: str, max_len: int = 140, repeat_stop: int = 6,
                 no_repeat_ngram: int = 3, top_k: int = 50, repetition_penalty: float = 1.2) -> str:
    """
    Uses no-repeat n-gram, repetition penalty, and top-k to reduce loops.
    Then removes duplicate transitions in post-processing.
    """
    # Encode input once
    seq = tokenizer.texts_to_sequences([prompt])
    seq = pad_sequences(seq, maxlen=max_encoder_len, padding="post")
    enc_outs, h, c = encoder_model.predict(seq, verbose=0)

    # Start with <start>
    target_id = start_id
    out_ids, out_tokens = [], []
    same_count, last_token = 0, None

    ban_ids = [0]  # pad
    if start_id is not None: ban_ids.append(start_id)
    if unk_id   is not None: ban_ids.append(unk_id)

    for _ in range(max_len):
        token_probs, h, c = decoder_model.predict(
            [np.array([[target_id]]), h, c, enc_outs], verbose=0
        )
        probs = token_probs[0, -1, :]

        next_id = _pick_with_constraints(
            probs=probs,
            prev_ids=out_ids,
            recent_window=30,
            repetition_penalty=repetition_penalty,
            no_repeat_ngram=no_repeat_ngram,
            ban_ids=ban_ids,
            top_k=top_k
        )

        # stopping
        if end_id is not None and next_id == end_id:
            break
        if next_id == last_token:
            same_count += 1
            if same_count >= repeat_stop:
                break
        else:
            same_count = 0
        last_token = next_id

        out_ids.append(next_id)
        tok = idx2word.get(next_id, "")
        if tok:
            out_tokens.append(tok)

        target_id = next_id

    raw_text = _clean_join(out_tokens)
    final_text = _dedup_transitions(raw_text)
    return final_text

# Quick check
print(generate_fsm("fsm description: odd number of 1's"))
